# Object internals
Every object in python contains the attribute `__dict__`, it holds all object attributes and their values. The content of the `__dict__` can be **read**, **updated**, **insertd**, and **deleted** like any other python dictionary.

On the other hand, the direct modification of the `__dict__` attribute is frown uppon. The recommended way is to use the build-in functions: `getattr`, `hasattr`, `delattr`, and`setattr`. **This methods are only invoked when the attributes are requested using the dot operator**.

In [71]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f'{self.__class__.__name__}(x={self.x}, y={self.y})'

In [3]:
v = Vector(5, 3)
dir(v)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'x',
 'y']

In [4]:
v.__dict__

{'x': 5, 'y': 3}

In [5]:
type(v.__dict__)

dict

In [6]:
v.__dict__['x']

5

In [7]:
v.__dict__['x'] = 7

In [8]:
v.x

7

In [9]:
del v.__dict__['x']

In [10]:
v.x

AttributeError: 'Vector' object has no attribute 'x'

In [11]:
v.__dict__['x'] = 'a'

In [12]:
v.x

'a'

In [13]:
'y' in v.__dict__

True

In [14]:
getattr(v, 'y')

3

In [15]:
hasattr(v, 'x')

True

In [17]:
delattr(v, 'x')

In [18]:
hasattr(v, 'x')

False

In [19]:
setattr(v, 'y', 9)

In [20]:
v.y

9

In [27]:
class GenericVector:
    def __init__(self, **kwargs):
        self.__dict__.update(**kwargs)
    
    def __repr__(self):
        coordinates = ', '.join(f'{k}={self.__dict__[k]}' for k in self.__dict__)
        return f'{self.__class__.__name__}({coordinates})'

In [30]:
GenericVector(a = 3, b = 4, c = 5)

GenericVector(a=3, b=4, c=5)

In [31]:
dir(_)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'a',
 'b',
 'c']

In [32]:
class GenericVector:
    def __init__(self, **kwargs):
        private_coordinates = {'_' + k:v for k, v in kwargs.items()}
        self.__dict__.update(**private_coordinates)
    
    def __repr__(self):
        coordinates = ', '.join(f'{k[1:]}={self.__dict__[k]}' for k in self.__dict__)
        return f'{self.__class__.__name__}({coordinates})'

In [33]:
GenericVector(a = 3, b = 4, c = 5)

GenericVector(a=3, b=4, c=5)

In [34]:
dir(_)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_a',
 '_b',
 '_c']

## Differences between `__getattr__` and `__getattribute__`
`__getattribute__` is the method that **all** attribute/properly lookups will call. `__getattr__` is invoked **after** an attribute/property lookup has not been found by a normal lookup.

In [43]:
class GenericVector:
    def __init__(self, **kwargs):
        private_coordinates = {'_' + k:v for k, v in kwargs.items()}
        self.__dict__.update(**private_coordinates)
    
    def __getattr__(self, name):
        print(f'name={name}')
    
    def __repr__(self):
        coordinates = ', '.join(f'{k[1:]}={self.__dict__[k]}' for k in self.__dict__)
        return f'{self.__class__.__name__}({coordinates})'

In [44]:
gv = GenericVector(a = 3, b = 4, c = 5)
gv.a

name=a


In [42]:
gv._a

3

In [45]:
class GenericVector:
    def __init__(self, **kwargs):
        private_coordinates = {'_' + k:v for k, v in kwargs.items()}
        self.__dict__.update(**private_coordinates)
    
    def __getattr__(self, name):
        private_name = '_' + name
        return getattr(self, private_name)
    
    def __repr__(self):
        coordinates = ', '.join(f'{k[1:]}={self.__dict__[k]}' for k in self.__dict__)
        return f'{self.__class__.__name__}({coordinates})'

In [46]:
gv = GenericVector(a = 3, b = 4, c = 5)
gv.a

3

In [47]:
gv.a = 10 # We don't want to allow this!

In [48]:
gv.a

10

In [49]:
dir(gv)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_a',
 '_b',
 '_c',
 'a']

In [50]:
# That's why we didn't want to allow that

In [51]:
gv.x # What happens when we request a non existing value?

RecursionError: maximum recursion depth exceeded while calling a Python object

In [52]:
class GenericVector:
    def __init__(self, **kwargs):
        private_coordinates = {'_' + k:v for k, v in kwargs.items()}
        self.__dict__.update(**private_coordinates)
    
    def __getattr__(self, name):
        private_name = '_' + name
        if not hasattr(self, private_name):
            raise AttributeError('{!r} object has no attribute {!r}'.format(self.__class__, name))
        return getattr(self, private_name)
    
    def __repr__(self):
        coordinates = ', '.join(f'{k[1:]}={self.__dict__[k]}' for k in self.__dict__)
        return f'{self.__class__.__name__}({coordinates})'

In [54]:
gv = GenericVector(a = 3, b = 4, c = 5)
gv.x # This will still fail because the `hasattr` internally uses the `__getattr__` of the object

RecursionError: maximum recursion depth exceeded while calling a Python object

In [55]:
class GenericVector:
    def __init__(self, **kwargs):
        private_coordinates = {'_' + k:v for k, v in kwargs.items()}
        self.__dict__.update(**private_coordinates)
    
    def __getattr__(self, name):
        private_name = '_' + name
        if private_name not in self.__dict__:
            raise AttributeError('{!r} object has no attribute {!r}'.format(self.__class__, name))
        return getattr(self, private_name)
    
    def __repr__(self):
        coordinates = ', '.join(f'{k[1:]}={self.__dict__[k]}' for k in self.__dict__)
        return f'{self.__class__.__name__}({coordinates})'

In [56]:
gv = GenericVector(a = 3, b = 4, c = 5)
gv.x

AttributeError: <class '__main__.GenericVector'> object has no attribute 'x'

In [57]:
class GenericVector:
    def __init__(self, **kwargs):
        private_coordinates = {'_' + k:v for k, v in kwargs.items()}
        self.__dict__.update(**private_coordinates)
    
    def __getattr__(self, name):
        private_name = '_' + name
        # Python is more about, ask for forgiveness than ask for permission, so...
        try:
            return self.__dict__[private_name]
        except KeyError:
            raise AttributeError('{!r} object has no attribute {!r}'.format(self.__class__, name))
    
    def __repr__(self):
        coordinates = ', '.join(f'{k[1:]}={self.__dict__[k]}' for k in self.__dict__)
        return f'{self.__class__.__name__}({coordinates})'

In [59]:
gv = GenericVector(a = 3, b = 4, c = 5)
gv.x

AttributeError: <class '__main__.GenericVector'> object has no attribute 'x'

# Vars (build-in function)
There is a more **pythonic** way to access the attributes of an object. It is using the build-in function `vars`.

```python
vars(obj)['p'] = "Wololo"
```
Is equivalent to
```python
obt.__dict__['p'] = "Wololo"
```

# Build-in functions special cases
The build-in functions such as `repr` bypass the `getattribute` method. Therefore if you are wrapping an object and you call `wrapper.__repr__()` it will properly forward the call to the wrapped object, but if you call `repr(wrapper)` it will output the `repr` of the wrapping object. In order to avoid that, you would need to implement the wrapping object's `__repr__` method to forward the call to the wrapped object.

# Where are method stored?
We've already seen that attributes are stored in the `__dict__` attribute of the object, however the methods are stored in another `__dict__` inside the `__class__` attribute.

The `__class__.__dict__` dictionary is not a common dictionary, it is of type `mappingproxy` and it does not support item assignment. In order to add a new entry or modify an existing entry in the map, the `setattr` build-in function must be used.

In [73]:
v = Vector(x=3, y=7)

In [61]:
v.__dict__

{'x': 3, 'y': 7}

In [62]:
v.__class__

__main__.Vector

In [63]:
v.__class__.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Vector.__init__(self, x, y)>,
              '__repr__': <function __main__.Vector.__repr__(self)>,
              '__dict__': <attribute '__dict__' of 'Vector' objects>,
              '__weakref__': <attribute '__weakref__' of 'Vector' objects>,
              '__doc__': None})

In [74]:
v.__class__.__dict__['__repr__'](v)

'Vector(x=3, y=7)'

In [75]:
v.__class__.__dict__['wololo'] = lambda s, x: print(f'Hello, {x}')

TypeError: 'mappingproxy' object does not support item assignment

In [78]:
setattr(v.__class__, 'wololo', lambda s, x: print(f'Hello, {x}')) # s is the self parameter in the method

In [77]:
v.wololo('Oscar')

Hello, Oscar
